In [1]:
import json
import time
from io import StringIO

import requests
import pandas as pd

from bs4 import BeautifulSoup

In [2]:
token = 'AgAAAAASqca9AAaElUQK1nKPWUfJuGs9In4AJPQ'

In [57]:
COUNTER_ID = 19249012
DATE1 = '2018-01-01'
DATE2 = '2021-03-31'
SOURCE = 'visits'
FIELDS = ['ym:s:visitID', 'ym:s:dateTime', 'ym:s:isNewUser','ym:s:visitDuration',
          'ym:s:startURL', 'ym:s:clientID','ym:s:productsPurchaseID', 'ym:s:lastTrafficSource']

In [58]:
PARAM = {
        'date1': DATE1,
        'date2':DATE2,
        'source':SOURCE,
        'fields':','.join(FIELDS)
}

## Оценка возможности создания запроса
Метода evaluate оценивает возможность создания запроса логов по его примерному размеру.

In [59]:
URL = 'https://api-metrika.yandex.ru/management/v1/counter/{counter_id}/logrequests/evaluate?'\
        .format(counter_id=COUNTER_ID)

In [60]:
r = requests.get(URL,headers={'Authorization': 'OAuth {0}'.format(token)},params=PARAM)

In [61]:
print(r.text)

{"log_request_evaluation":{"possible":true,"max_possible_day_quantity":86638}}


## Создание запроса логов
Метод logrequests - отправка к API Яндекс.Метрики информации о том, какой запрос хочется получить, чтобы сервера Яндекс.Метрики начали его готовить.

In [66]:
LOGREQUEST_URL = 'https://api-metrika.yandex.ru/management/v1/counter/{counter_id}/logrequests?'\
        .format(counter_id=COUNTER_ID)

In [67]:
logrequest = requests.post(LOGREQUEST_URL,headers={'Authorization': 'OAuth {0}'.format(token)},params=PARAM)

In [68]:
data = json.loads(logrequest.text)

In [69]:
data

{'log_request': {'request_id': 16691361,
  'counter_id': 19249012,
  'source': 'visits',
  'date1': '2018-01-01',
  'date2': '2021-03-31',
  'fields': ['ym:s:visitID',
   'ym:s:dateTime',
   'ym:s:isNewUser',
   'ym:s:visitDuration',
   'ym:s:startURL',
   'ym:s:clientID',
   'ym:s:productsPurchaseID',
   'ym:s:lastTrafficSource'],
  'status': 'created',
  'attribution': 'LASTSIGN'}}

ID запроса:

In [72]:
request_id = data['log_request']['request_id']

Отправка запроса с ID:

In [73]:
LOGREQUEST_URL_WITH_ID = 'https://api-metrika.yandex.ru/management/v1/counter/{counter_id}/logrequest/{request_id}'\
        .format(counter_id=COUNTER_ID,request_id=request_id)

In [74]:
logrequest_with_id = requests.get(LOGREQUEST_URL_WITH_ID,headers={'Authorization': 'OAuth {0}'.format(token)})

Статус:

In [119]:
log_status = json.loads(logrequest_with_id.text)['log_request']['status']

In [ ]:
while log_status != 'processed':
    time.sleep(60)
    if log_status == 'processed':
        print('READY!!!')
    else:
        print('trying')

## Загружаем лог

Просмотр полученных частей:

In [124]:
parts = json.loads(logrequest_with_id.text)['log_request']['parts']

In [126]:
parts

[{'part_number': 0, 'size': 63130677}, {'part_number': 1, 'size': 63103146}]

Сведение номера частей из словарей в один список:

In [127]:
parts_list = []
for part in parts:
        parts_list.append(part['part_number'])

In [128]:
parts_list

[0, 1]

In [129]:
tmp_dfs = []
for part_num in parts_list:
        URL = 'https://api-metrika.yandex.ru/management/v1/counter/{counter_id}/logrequest/{request_id}/part/{part}/download' \
                        .format(
                                counter_id=COUNTER_ID,
                                request_id=request_id,
                                part=part_num
                        )

        r = requests.get(URL,headers={'Authorization': 'OAuth {0}'.format(token)})
        if r.status_code == 200:
                tmp_df = pd.read_csv(StringIO(r.text), sep = '\t')
                tmp_dfs.append(tmp_df)

In [ ]:
logs_df = pd.concat(tmp_dfs)

## Анализ

Переименование столбцов:

In [132]:
logs_df.columns = ['ID_визита','Дата','Является_новым_посетителем','Длительность_визита',
              'Страница_входа','ID_клиента', 'ID_покупки','Источник_визита']

Перевод значений дат в формат даты:

In [144]:
logs_df['Дата'] = pd.to_datetime(logs_df['Дата'], format='%Y-%m-%d').dt.date

### Совершали ли покупки люди, которые впервые пришли через блог

Срез по визитам на блог:

In [152]:
blog_filter = logs_df['Страница_входа'].str.contains(pat='beauty-blog', regex=True)

In [153]:
visits_on_blog = logs_df[blog_filter]

In [176]:
visits_on_blog.shape

(56955, 8)

Срез по тем, кто пришел на блог впервые:

In [157]:
new_clients_from_blog = visits_on_blog[visits_on_blog['Является_новым_посетителем'] == 1]

In [179]:
new_clients_from_blog.shape

(51260, 8)

In [159]:
new_clients_from_blog_id_list = new_clients_from_blog['ID_клиента'].to_list()

Срез общего датафрейма по тем ID клиентов, которые впервые попали на сайт через блог:

In [162]:
new_clients_from_blog_filter = logs_df['ID_клиента'].isin(new_clients_from_blog_id_list)

In [174]:
clients_came_from_blog = logs_df[new_clients_from_blog_filter]

Срез тех, кто совершил покупку:

In [182]:
clients_came_from_blog_made_purchase = clients_came_from_blog[clients_came_from_blog['ID_покупки'] != '[]']

#### Анализ полученной информации о купивших

Сколько покупок среди таких клиентов:

In [216]:
clients_came_from_blog_made_purchase['ID_покупки'].nunique()

36

**Всего покупок:** 36

Когда первая и последняя покупка:

In [191]:
clients_came_from_blog_made_purchase['Дата'].min()

datetime.date(2018, 7, 9)

In [192]:
clients_came_from_blog_made_purchase['Дата'].max()

datetime.date(2021, 3, 23)

**Первая:** 9 июля 2018 <br>
**Последняя:** 23 марта 2021

Сколько клиентов купили сразу, придя на блог:

In [217]:
clients_came_from_blog_made_purchase[clients_came_from_blog_made_purchase['Является_новым_посетителем'] == 1].shape

25

**Купили сразу:** 25 клиентов

In [219]:
clients_came_from_blog_made_purchase['ID_покупки'].nunique()

36

In [211]:
clients_came_from_blog['ID_клиента'].nunique()

50675

### Совершали ли покупки люди, которые впервые пришли через FAQ

Срез по визитам на FAQ:

In [220]:
faq_filter = logs_df['Страница_входа'].str.contains(pat='faq', regex=True)

In [221]:
visits_on_faq = logs_df[faq_filter]

In [222]:
visits_on_faq.shape

(83435, 8)

Срез по тем, кто пришел на блог впервые:

In [223]:
new_clients_from_faq = visits_on_faq[visits_on_faq['Является_новым_посетителем'] == 1]

In [224]:
new_clients_from_faq.shape

(76414, 8)

In [225]:
new_clients_from_faq_id_list = new_clients_from_faq['ID_клиента'].to_list()

Срез общего датафрейма по тем ID клиентов, которые впервые попали на сайт через блог:

In [226]:
new_clients_from_faq_filter = logs_df['ID_клиента'].isin(new_clients_from_faq_id_list)

In [227]:
clients_came_from_faq = logs_df[new_clients_from_faq_filter]

Срез тех, кто совершил покупку:

In [228]:
clients_came_from_faq_made_purchase = clients_came_from_faq[clients_came_from_faq['ID_покупки'] != '[]']

#### Анализ полученной информации о купивших

Сколько покупок среди таких клиентов:

In [229]:
clients_came_from_faq_made_purchase['ID_покупки'].nunique()

24

**Всего покупок:** 24

Когда первая и последняя покупка:

In [231]:
clients_came_from_faq_made_purchase['Дата'].min()

datetime.date(2018, 4, 13)

In [232]:
clients_came_from_faq_made_purchase['Дата'].max()

datetime.date(2020, 12, 15)

**Первая:** 13 апреля 2018 <br>
**Последняя:** 15 декабря 2020

Сколько клиентов купили сразу, придя на блог:

In [233]:
clients_came_from_faq_made_purchase[clients_came_from_faq_made_purchase['Является_новым_посетителем'] == 1].shape

(9, 8)

**Купили сразу:** 9 клиентов

In [234]:
clients_came_from_faq_made_purchase['ID_покупки'].nunique()

24

In [235]:
clients_came_from_faq['ID_клиента'].nunique()

75600

### Совершали ли покупки люди, которые впервые пришли через Отзывы

Срез по визитам на Отзывы:

In [236]:
reviews_filter = logs_df['Страница_входа'].str.contains(pat='otzyvy-o-kosmetike', regex=True)

In [237]:
visits_on_reviews = logs_df[reviews_filter]

In [238]:
visits_on_reviews.shape

(13498, 8)

Срез по тем, кто пришел на Отзывы впервые:

In [239]:
new_clients_from_reviews = visits_on_reviews[visits_on_reviews['Является_новым_посетителем'] == 1]

In [240]:
new_clients_from_reviews.shape

(11734, 8)

In [241]:
new_clients_from_reviews_id_list = new_clients_from_reviews['ID_клиента'].to_list()

Срез общего датафрейма по тем ID клиентов, которые впервые попали на сайт через Отзывы:

In [242]:
new_clients_from_reviews_filter = logs_df['ID_клиента'].isin(new_clients_from_reviews_id_list)

In [243]:
clients_came_from_reviews = logs_df[new_clients_from_reviews_filter]

Срез тех, кто совершил покупку:

In [244]:
clients_came_from_reviews_made_purchase = clients_came_from_reviews[clients_came_from_reviews['ID_покупки'] != '[]']

#### Анализ полученной информации о купивших

Сколько покупок среди таких клиентов:

In [245]:
clients_came_from_reviews_made_purchase['ID_покупки'].nunique()

22

**Всего покупок:** 22

Когда первая и последняя покупка:

In [247]:
clients_came_from_reviews_made_purchase['Дата'].min()

datetime.date(2019, 11, 10)

In [248]:
clients_came_from_reviews_made_purchase['Дата'].max()

datetime.date(2021, 3, 7)

**Первая:** 10 ноября 2019 <br>
**Последняя:** 7 марта 2021

Сколько клиентов купили сразу, придя на Отзывы:

In [249]:
clients_came_from_reviews_made_purchase[clients_came_from_reviews_made_purchase['Является_новым_посетителем'] == 1].shape

(10, 8)

**Купили сразу:** 10 клиентов

In [234]:
clients_came_from_reviews_made_purchase['ID_покупки'].nunique()

24

In [250]:
clients_came_from_reviews['ID_клиента'].nunique()

11604